In [14]:
from ref import NAMES

from helpers.scrape import scrape_standings
import numpy as np

In [15]:
standings = scrape_standings()

/Users/MB13693/Desktop/Perso/pl-preds/helpers/scrape.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  standings = pd.read_html(str(table))[0]


In [17]:
import random

for name in NAMES:

    with open(f'predictions/{name.lower()}.txt', 'r') as f:
        preds = [line.strip() for line in f.readlines()]

    random.shuffle(preds)

    standings[f'pred_{name.lower()}'] = preds

    standings[f'diff_{name.lower()}'] = [np.abs(np.where(standings[f'pred_{name.lower()}'] == tm)[0][0] - idx) for tm, idx in zip(standings.actual, standings.index)]
    standings[f'perf_{name.lower()}'] = (standings[f'diff_{name.lower()}'] == 0).astype(int)

In [18]:
for name in ['martin', 'thomas', 'alexis'] :
    standings[f'diff_{name}'] = [np.abs(np.where(standings[f'pred_{name}'] == tm)[0][0] - idx) for tm, idx in zip(standings.actual, standings.index)]
    standings[f'perf_{name}'] = (standings[f'diff_{name}'] == 0).astype(int)

In [38]:
total_diff = standings['diff_martin'].sum()
total_perf = standings['perf_martin'].sum()

In [32]:
worst = standings.sort_values('diff_martin')['actual'].values[-1]

In [33]:
perfects = standings[standings['perf_martin'] == 1]

In [36]:
', '.join(perfects['actual'].tolist())

"Brentford, Nott'ham Forest"

In [ ]:
def plot(scores, files, sorted_names) :
    
    time = [datetime.strptime(file[8:18], '%Y-%m-%d') for file in files]

    _, ax = plt.subplots(figsize = (18, 9), facecolor = 'black')

    plt.gca().set_facecolor('black')
    plt.plot(time, gaussian_filter1d(scores[:,0], 0.4), c = 'green', marker = 'o', linewidth = 5, markersize = 10, label = 'Alexis', zorder = 3 - np.where(np.array(sorted_names) == 'Alexis')[0][0])
    plt.plot(time, gaussian_filter1d(scores[:,1], 0.4), c = 'red',   marker = 'o', linewidth = 5, markersize = 10, label = 'Martin', zorder = 3 - np.where(np.array(sorted_names) == 'Martin')[0][0])
    plt.plot(time, gaussian_filter1d(scores[:,2], 0.4), c = 'blue',  marker = 'o', linewidth = 5, markersize = 10, label = 'Thomas', zorder = 3 - np.where(np.array(sorted_names) == 'Thomas')[0][0])
    plt.plot(time, scores[:,0], c = 'green', linewidth = 1, alpha = 0.5, zorder = - np.where(np.array(sorted_names) == 'Alexis')[0][0])
    plt.plot(time, scores[:,1], c = 'red',   linewidth = 1, alpha = 0.5, zorder = - np.where(np.array(sorted_names) == 'Martin')[0][0])
    plt.plot(time, scores[:,2], c = 'blue',  linewidth = 1, alpha = 0.5, zorder = - np.where(np.array(sorted_names) == 'Thomas')[0][0])

    plt.xticks(time, pd.Series(time).apply(lambda x: datetime.strftime(x, '%d %b')), color = 'white')
    plt.yticks(np.arange(0, 120, 20), np.arange(0, 120, 20), color = 'white')
    plt.ylabel('Score', size = 32, color = 'white')
    plt.title('2023-24 PREMIER LEAGUE PREDICTIONS: scores over time', size = 36, color = 'white')

    ax.tick_params(axis = 'both', labelsize = 24)
    plt.grid(axis = 'y', color = 'white', linestyle = ':', alpha = 0.8)
    plt.ylim([0, 120])

    ax.spines['left'].set_color('white'); ax.spines['right'].set_color('white')
    ax.spines['bottom'].set_color('white'); ax.spines['top'].set_color('white') 

    lgd = plt.legend(edgecolor = 'white', labelcolor = 'white', fontsize = 24, loc = 'lower right')
    lgd.get_frame().set_facecolor('black')

    path = f'plots/plot-{str(date.today())}.jpg'
    plt.savefig(path, dpi = 400)
    plt.close()

    return path

def emailResults(scores, path, prog, sorted_names, sorted_scores) :

    # Create a MIMEMultipart object
    msg = MIMEMultipart()

    # Set the sender and recipient addresses
    l = scores[-1,:]
    leaders = list(np.array(names)[(np.argwhere(l == np.max(l)).flatten())])
    mult = len(leaders) > 1
    subject = 'PREMIER LEAGUE PREDICTIONS UPDATE: ' + f'{" & ".join(leaders)} {mult*"are"}{(1 - mult) * "is"} leading the way with {np.max(l)} points !'
    msg['Subject'] = subject

    my_mail = 'martinbog19@gmail.com'
    recipients = [my_mail, 'tbogaert19@gmail.com', 'alexis.bogaert2309@gmail.com']

    # Create the email body as HTML
    html = f"""
    <html>
    <body>
    <p>Rhoya la mif,</p>
    <p>Leaderboard after completion of {prog}% of the 2023-24 Premier League season:</p>
    <p style="margin-left: 20px;">1. {sorted_names[0]}:  {sorted_scores[0]} pts<br>
    <p style="margin-left: 20px;">2. {sorted_names[1]}:  {sorted_scores[1]} pts<br>
    <p style="margin-left: 20px;">3. {sorted_names[2]}:  {sorted_scores[2]} pts</p>
    <p><img src="cid:image1" alt="Image"></p>
    </body>
    </html>
    """
    msg.attach(MIMEText(html, 'html'))

    # Read and attach the image
    with open(path, 'rb') as img_file:
        img = MIMEImage(img_file.read())
        img.add_header('Content-ID', '<image1>')
        msg.attach(img)

    with open('key.txt', 'r') as k :
        password = k.read()

    # Set up the SMTP server and send the email
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(my_mail, password)
        server.sendmail(my_mail, recipients, msg.as_string())
        print('Email sent successfully!')
    except Exception as e:
        print('Error sending email: ', str(e))
    finally:
        server.quit()



def main() :

    prog = createScoresDf()
    
    files = sorted(os.listdir('results/'))
    scores, perfects = np.zeros((len(files), 3)), np.zeros((len(files), 3))

    for i, file in enumerate(files) :
        for j, name in enumerate(names) :

            score, perfect = getScorePerfects(name, 'results/' + file)
            scores[i,j]   = score
            perfects[i,j] = perfect
    
    current_scores = list(scores[-1,:])
    combined = list(zip(names, current_scores))
    sorted_combined = sorted(combined, key = lambda x: x[1], reverse = True)
    sorted_names, sorted_scores = zip(*sorted_combined)

    path = plot(scores, files, sorted_names)
    emailResults(scores, path, prog, sorted_names, sorted_scores)


if __name__ == '__main__' :


In [48]:
import os
import requests

account_sid = 'ACa7f0f17d7d08787859bb5db69b6f098a'
auth_token = 'fb21f2284953b11329161ca22f6ce916'
from_number = "+14155238886"
to_number = "+41792925629"

message = "Salut c'est Martin"

url = f"https://api.twilio.com/2010-04-01/Accounts/{account_sid}/Messages.json"
data = {
    "From": f"whatsapp:{from_number}",
    "To": f"whatsapp:{to_number}",
    "Body": message,
}

response = requests.post(url, data=data, auth=(account_sid, auth_token))
print("Status:", response.status_code)
print("Response:", response.text)


Status: 201
Response: {"account_sid": "ACa7f0f17d7d08787859bb5db69b6f098a", "api_version": "2010-04-01", "body": "Salut c'est Martin", "date_created": "Mon, 11 Aug 2025 16:14:37 +0000", "date_sent": null, "date_updated": "Mon, 11 Aug 2025 16:14:37 +0000", "direction": "outbound-api", "error_code": null, "error_message": null, "from": "whatsapp:+14155238886", "messaging_service_sid": null, "num_media": "0", "num_segments": "1", "price": null, "price_unit": null, "sid": "SM0c85ef00ccec667accf1b3f135eac75b", "status": "queued", "subresource_uris": {"media": "/2010-04-01/Accounts/ACa7f0f17d7d08787859bb5db69b6f098a/Messages/SM0c85ef00ccec667accf1b3f135eac75b/Media.json"}, "to": "whatsapp:+41792925629", "uri": "/2010-04-01/Accounts/ACa7f0f17d7d08787859bb5db69b6f098a/Messages/SM0c85ef00ccec667accf1b3f135eac75b.json"}


In [49]:
# Download the helper library from https://www.twilio.com/docs/python/install
from twilio.rest import Client

client = Client(account_sid, auth_token)

message = client.messages.create(
    body="Hello there!",
    from_="whatsapp:+14155238886",
    to="whatsapp:+18574520957",
)

print(message.body)

Hello there!


In [ ]:
os.getenv()